<a href="https://colab.research.google.com/github/bitanb1999/TalentSumoAI/blob/main/GPT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 4.2 MB/s 
     |████████████████████████████████| 895 kB 41.3 MB/s 
     |████████████████████████████████| 77 kB 6.1 MB/s 
     |████████████████████████████████| 6.6 MB 33.1 MB/s 
     |████████████████████████████████| 596 kB 50.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
#for reproducability
SEED = 34

#maximum number of words in output text
MAX_LEN = 70

###I. Intro
A language model is a machine learning model that can look at part of a sentence and predict the next word/sequence of words. Much like the autofill features on your iPhone/Android, GPT-2 is capable of next word prediction on a much larger and more sophisticated scale. For reference, the smallest available GPT-2 has 117 million parameters, whereas the largest one (invisible to the public) has over 1.5 billion parameters. The largest one available for public use is half the size of their main GPT-2 model

😊 Transformers makes it very easy to import this model with both PyTorch and TensorFlow - in this notebook we will be using TensorFlow but it is just as easy in PyTorch. Both the model and its Tokenizer can be imported from the transformers library that anyone can get by typing !pip install transformers. Let's see just how simple it is to generate text with a neural network. We begin with our input sequence:

In [10]:
input_sequence = "I am enthusiastic,diligent and hardowking. I am very passionate about my work and am a very good communicator. Data science"

In [4]:
#get transformers
from transformers import TFGPT2LMHeadModel, GPT2Tokenizer

#get large GPT2 tokenizer and GPT2 model
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-large")
GPT2 = TFGPT2LMHeadModel.from_pretrained("gpt2-large", pad_token_id=tokenizer.eos_token_id)
GPT2.summary()

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/666 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.88G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2-large.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Model: "tfgpt2lm_head_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLaye  multiple                 774030080 
 r)                                                              
                                                                 
Total params: 774,030,080
Trainable params: 774,030,080
Non-trainable params: 0
_________________________________________________________________


In [5]:
#get deep learning basics
import tensorflow as tf
tf.random.set_seed(SEED)

In [11]:
# encode context the generation is conditioned on
input_ids = tokenizer.encode(input_sequence, return_tensors='tf')
# set return_num_sequences > 1
beam_outputs = GPT2.generate(
    input_ids, 
    max_length = MAX_LEN, 
    num_beams = 5, 
    no_repeat_ngram_size = 2, 
    num_return_sequences = 5, 
    early_stopping = True
)

print('')
print("Output:\n" + 50 * '-')

# now we have 3 output sequences
for i, beam_output in enumerate(beam_outputs):
      print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))


Output:
--------------------------------------------------
0: I am enthusiastic,diligent and hardowking. I am very passionate about my work and am a very good communicator. Data science is one of the most exciting fields in the world, and I want to be a part of it.

What are you working on at the moment?


My current project is to build a
1: I am enthusiastic,diligent and hardowking. I am very passionate about my work and am a very good communicator. Data science is one of the most exciting fields in the world, and I want to be a part of it.

What are you working on at the moment?


I'm currently working with a team
2: I am enthusiastic,diligent and hardowking. I am very passionate about my work and am a very good communicator. Data science is one of the most exciting fields in the world, and I want to be a part of it.

What are you working on at the moment?


My current project is to build an
3: I am enthusiastic,diligent and hardowking. I am very passionate about my work and am a ve

In [12]:
# use temperature to decrease the sensitivity to low probability candidates
sample_output = GPT2.generate(
                             input_ids, 
                             do_sample = True, 
                             max_length = MAX_LEN, 
                             top_k = 0, 
                             temperature = 0.8
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens = True))

Output:
----------------------------------------------------------------------------------------------------
I am enthusiastic,diligent and hardowking. I am very passionate about my work and am a very good communicator. Data science is an interesting field for me because it involves a lot of problems that most people would not even consider. Right now, I have a chance to work in space and I can communicate with the smartest machine ever


In [13]:
#sample only from 80% most likely words
sample_output = GPT2.generate(
                             input_ids, 
                             do_sample = True, 
                             max_length = MAX_LEN, 
                             top_p = 0.8, 
                             top_k = 0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens = True), '...')

Output:
----------------------------------------------------------------------------------------------------
I am enthusiastic,diligent and hardowking. I am very passionate about my work and am a very good communicator. Data science and I complement each other extremely well and will be using them to grow the company. You will be working closely with both me and Dr. Nakanishi (Dr.Hossein). We have a ...


In [14]:
#combine both sampling techniques
sample_outputs = GPT2.generate(
                              input_ids,
                              do_sample = True, 
                              max_length = 2*MAX_LEN,                              #to test how long we can generate and it be coherent
                              #temperature = .7,
                              top_k = 50, 
                              top_p = 0.85, 
                              num_return_sequences = 5
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
    print("{}: {}...".format(i, tokenizer.decode(sample_output, skip_special_tokens = True)))
    print('')

Output:
----------------------------------------------------------------------------------------------------
0: I am enthusiastic,diligent and hardowking. I am very passionate about my work and am a very good communicator. Data science has become a popular and lucrative career path for me. I love helping organizations get ahead in their digital transformation efforts.

How long have you been doing this?

Since 2010. I was one of the original co-founders and lead engineer for a product in a data science team that I helped build called Big Data Analytics (BDaaS). I left this job and joined Big Data Analytics as the lead engineer in 2016. My primary focus is on data science and data visualization.

How did you get your start?

My...

1: I am enthusiastic,diligent and hardowking. I am very passionate about my work and am a very good communicator. Data science and data analytics are my lifeblood. The work I have done is in a variety of different fields and I have developed a variety of skil